# Variable Definitions demo for creators and owners of Variable definitions
Creator team of is set as variable owner. Only owner can edit a Variable defintion.

We recommend using Vardef Model for working with variable definitions to avoid json issues and get help from available options. But it is also possible to use string json input.

Examples models: 
- Draft()
- UpdateDraft()
- Patch
- Owner



In [3]:
from datetime import date

from dapla_metadata.variable_definitions.generated.vardef_client.models.contact import (
    Contact,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.draft import (
    Draft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.update_draft import (
    UpdateDraft,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.validity_period import (
    ValidityPeriod,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.variable_status import (
    VariableStatus,
)
from dapla_metadata.variable_definitions.vardef import Vardef


## Environment

In [4]:

from dapla_metadata.variable_definitions.config import get_active_group
from dapla_metadata.variable_definitions.config import get_oidc_token
from dapla_metadata.variable_definitions.config import get_vardef_host

get_active_group()
get_oidc_token()
get_vardef_host()


'http://localhost:8080'

## Drafts

### Create a draft
Can produce a 409 CONFLICT if the `short_name` already exists.

In [21]:
draft = Draft(
    name = {
        "nb": "test_navn",
        "nn": "test_namn",
        "en": "test_name",
    },
    short_name= "latter10",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

In [22]:
my_draft = Vardef.create_draft(draft)

### Exception creating draft
Trigger an exception with invalid Klass codes

In [10]:
invalid_draft = Draft(
    name = {
        "nb": "test_navn",
        "nn": "test_namn",
        "en": "test_name",
    },
    short_name= "lan",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "haha",
    ],
    subject_fields=[
        "a",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

Vardef.create_draft(invalid_draft)

#### Check details Variabvle definition

In [44]:
# Examples how to check details
import pprint


print(my_draft.definition)
print(f"Id: {my_draft.id}")
print(f"Owner: {my_draft.owner}")
print(Vardef.get_variable_definition(my_draft.id).created_at)

# Fields in variable are not automatical updated. You can either update like this:
my_draft = Vardef.get_variable_definition(my_draft.id)

# Or use Vardef.get_variable_definition(my_draft.id) directly to check values


{'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}
Id: ycxL8g7B
Owner: {'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}
2024-12-19 08:46:31.511000


### Edit a draft

This operation must be performed on Variable Definitions with status DRAFT.

In [ ]:
# Update contact information we recc
my_contact = Contact(
    title={
        "nb": "res",
        "nn": "res",
        "en": "res",
        },
    email="sibby@ssb.no",
)
update_contact = UpdateDraft(
    contact=my_contact,
)
my_draft.update_draft(update_draft=update_contact)
print(Vardef.get_variable_definition(my_draft.id))

In [13]:
# Test draft exceptions invalid short_name
invalid_short_name = UpdateDraft(
    short_name="_)(45)",
)
my_draft.update_draft(invalid_short_name)



ValidationError: 1 validation error for UpdateDraft
short_name
  Value error, must validate the regular expression /^[a-z0-9_]{3,}$/ [type=value_error, input_value='_)(45)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

### Migrate a legacy definition from Vardok to Vardef

Note: This will raise an error if the short name already exists.

In [15]:
# Migrate from Vardok exception invalid id
Vardef.migrate_from_vardok("21")

VardefClientException: Status 404: Vardok id 21 not found

In [16]:
# Save variable Vardok
my_vardok_draft = Vardef.migrate_from_vardok("90")
print(my_vardok_draft)

{
  "id": "D3L6wTZW",
  "patch_id": 1,
  "name": {
    "nb": "Spesifisert registreringstype",
    "nn": null,
    "en": null
  },
  "short_name": "spes_reg_type",
  "definition": {
    "nb": "Variabelen viser kode for personens særforhold i folkeregistreringen",
    "nn": null,
    "en": null
  },
  "classification_reference": null,
  "unit_types": [
    "20"
  ],
  "subject_fields": [],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "DRAFT",
  "measurement_type": null,
  "valid_from": "1996-06-01",
  "valid_until": null,
  "external_reference_uri": "https://www.ssb.no/a/xml/metadata/conceptvariable/vardok/90",
  "comment": null,
  "related_variable_definition_uris": [],
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers"
    ]
  },
  "contact": null,
  "created_at": "2024-12-19T08:43:37.360808",
  "created_by": null,
  "last_updated_at": "2024-12-19T08:43:37.360808",
  "last_updated_by": null
}


### Delete a draft

This operation must be performed on Variable Definitions with status DRAFT.

In [17]:
my_vardok_draft.delete_draft()

'Variable D3L6wTZW safely deleted'

## Publish
When draft has been quality checked and the resulkt is as desired it is time to Publish the variable, either internal or external. Be aware: this process is irreversibel.

In [24]:
update_status = UpdateDraft(
    variable_status=VariableStatus.PUBLISHED_INTERNAL,
)

print(my_draft.update_draft(update_status))

VardefClientException: Status 405: The variable is published and cannot be updated with this method

## Patch
### Create a patch
#### Update owner
Owner can be extended or completly transferred to another team.  

#### Update owner by adding owner group

In [25]:
from dapla_metadata.variable_definitions.generated.vardef_client.models.owner import (
    Owner,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

new_owner = Patch(
    owner=Owner(
    team="dapla-felles",
    groups=[
        "dapla-felles-developers",
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(new_owner)
my_draft = Vardef.get_variable_definition(my_draft.id)
print(my_draft.owner)

{'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}


#### Update owner by replacing team and group
be aware of if you remove your team entirely from owner the variable will be forbidden. Feel free to test.But


In [20]:
replace_owner = Patch(
    owner=Owner(
    team="play-enhjoern-a",
    groups=[
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(replace_owner)
my_draft = Vardef.get_variable_definition(my_draft.id)
print(my_draft.owner)
my_draft.create_patch(Patch(unit_types=["01"]))

{'team': 'play-enhjoern-a', 'groups': ['play-enhjoern-a-developers']}


VardefClientException: Status 403: Forbidden

In [ ]:
#forbidden exception

In [ ]:
# change owner

In [26]:
# read details
Vardef.get_variable_definition(my_draft.id).subject_fields

['al']

In [30]:
patches = my_draft.list_patches()
for patch in patches:
    print(patch.patch_id)

1
2


## Validity period

In [31]:
# create validity period


invalid_validity_period = ValidityPeriod(
    name={
        "nb": "new name",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

ValidationError: 1 validation error for ValidityPeriod
definition
  Field required [type=missing, input_value={'name': {'nb': 'new name...tetime.date(2025, 4, 5)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [32]:
# validity period exceptions
# correct validity period
# check languages

valid_validity_period = ValidityPeriod(
    definition={
        "nb": "ny definisjon2",
        "nn": "ny definisjon2",
        "en": "new definition2",
    },
    valid_from=date(2040,4,5),
)
my_draft.create_validity_period(valid_validity_period)


CompletePatchOutput(id='ycxL8g7B', patch_id=4, name={'nb': 'test_navn', 'nn': 'test_namn', 'en': 'test_name'}, short_name='latter10', definition={'nb': 'ny definisjon2', 'nn': 'ny definisjon2', 'en': 'new definition2'}, classification_reference='91', unit_types=['01'], subject_fields=['al'], contains_special_categories_of_personal_data=False, variable_status=<VariableStatus.PUBLISHED_INTERNAL: 'PUBLISHED_INTERNAL'>, measurement_type=None, valid_from=datetime.date(2040, 4, 5), valid_until=None, external_reference_uri=None, comment=None, related_variable_definition_uris=None, owner={'team': 'dapla-felles', 'groups': ['dapla-felles-developers', 'play-enhjoern-a-developers']}, contact=None, created_at=datetime.datetime(2024, 12, 19, 8, 46, 31, 511883), created_by=None, last_updated_at=datetime.datetime(2024, 12, 19, 8, 46, 31, 511883), last_updated_by=None)

In [33]:
print(Vardef.get_variable_definition(my_draft.id))

{
  "id": "ycxL8g7B",
  "patch_id": 4,
  "name": {
    "nb": "test_navn",
    "nn": "test_namn",
    "en": "test_name"
  },
  "short_name": "latter10",
  "definition": {
    "nb": "ny definisjon2",
    "nn": "ny definisjon2",
    "en": "new definition2"
  },
  "classification_reference": "91",
  "unit_types": [
    "01"
  ],
  "subject_fields": [
    "al"
  ],
  "contains_special_categories_of_personal_data": false,
  "variable_status": "PUBLISHED_INTERNAL",
  "measurement_type": null,
  "valid_from": "2040-04-05",
  "valid_until": null,
  "external_reference_uri": null,
  "comment": null,
  "related_variable_definition_uris": null,
  "owner": {
    "team": "dapla-felles",
    "groups": [
      "dapla-felles-developers",
      "play-enhjoern-a-developers"
    ]
  },
  "contact": null,
  "created_at": "2024-12-19T08:46:31.511000",
  "created_by": null,
  "last_updated_at": "2024-12-19T08:46:31.511000",
  "last_updated_by": null
}


In [34]:
validity_periods = my_draft.list_validity_periods()
for period in validity_periods:
    print(period.valid_from)

2024-12-18
2040-04-05
